In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import pickle
import seaborn as sns
import skimage

import analyze
import evaluate
import characterize

In [ ]:
with open('../data/interim/instance_sets/particle_gt_instance_sets.pickle', 'rb') as f:
    particle_gt_instances = pickle.load(f)
with open('../data/interim/instance_sets/satellite_gt_instance_sets.pickle', 'rb') as f:
    satellite_gt_instances = pickle.load(f)

with open('../data/interim/instance_sets/particle_pred_instance_sets.pickle', 'rb') as f:
    particle_pred_instances = pickle.load(f)    
with open('../data/interim/instance_sets/satellite_pred_instance_sets.pickle', 'rb') as f:
    satellite_pred_instances = pickle.load(f)

In [ ]:
for group in [particle_pred_instances, satellite_pred_instances]: # fix for filepaths stored as tuples, TODO remove after bug fix
    for instance in group:
        instance.filepath = instance.filename[0]

In [ ]:
def instance_pair_matcher(A, B):
    """
    matches instance lists A and B based on their filenames
    """
    
    Bdict = {pathlib.Path(item.filepath).name : item for item in B}
    
    A_matched = []
    B_matched = []
    for item in A:
        x = Bdict.get(pathlib.Path(item.filepath).name, None)
        if x is not None:
            A_matched.append(item)
            B_matched.append(x)
    return A_matched, B_matched
    

In [ ]:
# only keep gt satellites with corresponding particle masks drawn
particle_gt_matched, satellite_gt_matched = instance_pair_matcher(particle_gt_instances, satellite_gt_instances)
# makes sure order of particle predictions is same as particle gt
particle_gt_matched, particle_pred_matched = instance_pair_matcher(particle_gt_matched, particle_pred_instances)
# only keep pred instances with corresponding satellites
particle_pred_matched, satellite_pred_matched = instance_pair_matcher(particle_pred_matched, satellite_pred_instances)

In [ ]:
for gt, pred in zip(particle_gt_matched, particle_pred_matched):
    pred.img = gt.img

In [ ]:
val_mask = [x.dataset_type=='Validation' for x in particle_gt_matched]

particle_gt_val = [x for x, b in zip(particle_gt_matched, val_mask) if b][0]
satellite_gt_val =[x for x, b in zip(satellite_gt_matched, val_mask) if b][0]
particle_pred_val = [x for x, b in zip(particle_pred_matched, val_mask) if b][0]
satellite_pred_val = [x for x, b in zip(satellite_pred_matched, val_mask) if b][0]


In [ ]:
psi_gt = characterize.powder_satellite_image(particle_gt_val, satellite_gt_val)
psi_pred = characterize.powder_satellite_image(particle_pred_val, satellite_pred_val)

psi_gt.compute_matches()
psi_pred.compute_matches()

for x in [psi_gt, psi_pred]:
    for y in [x.particles, x.satellites]:
        y.boxes =y.boxes.astype(np.int)

In [ ]:
psi_gt.matches['particle_satellite_match_idx'].keys()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,5), dpi=300)
for i, a in zip(np.random.choice(list(psi_gt.matches['particle_satellite_match_idx'].keys()), size=3, replace=False),
                 ax.ravel()):
    psi_gt.visualize_particle_with_satellites(i, a)
ax[0].set_title('ground truth')    
fig, ax = plt.subplots(1,3, figsize=(15,5), dpi=300)
for i, a in zip(np.random.choice(list(psi_pred.matches['particle_satellite_match_idx'].keys()), size=3, replace=False),
                 ax.ravel()):
    psi_pred.visualize_particle_with_satellites(i, a)
ax[0].set_title('predicted')

In [ ]:
gt_n = psi_gt.matches['particles_matched'].shape[0]
pred_n = psi_pred.matches['particles_matched'].shape[0]
print('Number of particles with satellites:\n\tgt: {}\n\tpred: {}\n\tratio: {:.3f}'.format(gt_n, pred_n, pred_n/gt_n))